In [4]:
# soup_temp = 1

In [5]:
def convert_to_hindi_number(input_str):
    english_numbers = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    hindi_numbers = ["०", "१", "२", "३", "४", "५", "६", "७", "८", "९"]

    result = []

    for char in input_str:
        if char.isdigit():
            result.append(hindi_numbers[english_numbers.index(char)])
        elif char == '-':
            result.append('-')  # Preserve hyphen
        else:
            result.append(char)  # Preserve other characters

    return ''.join(result)

In [6]:
import re

def remove_bg(input_text):
    pattern = re.compile(r'^BG \d+\.\d+-\d+\n:')
    result = re.sub(pattern, '', input_text)
    pattern = re.compile(r'^BG \d+\.\d+\n:')
    result = re.sub(pattern, '', result)
    return result

def remove_numbered_strings(input_text):
    pattern = re.compile(r'\n\d+\n')
    result = re.sub(pattern, '', input_text)
    return result

In [7]:
def find_verse(soup):
    if soup.find('div', {'id': 'originalVerse'}) != None:
        return '\n'.join([paragraph.get_text('\n', strip=True) for paragraph in soup.find('div', {'id': 'originalVerse'}).find_all('p')]).strip('\n')
    else:
        return "NONE"

In [8]:
def synonyms(soup):
    if soup.find('div', {'id': 'wordMeanings'}) != None and soup.find('div', {'id': 'wordMeanings'}).find('p') != None:
        return soup.find('div', {'id': 'wordMeanings'}).find('p').get_text('\n', strip=True).strip('\n')
    else:
        return "NONE"

In [9]:
def translation(soup):
    if soup.find('div', {'id': 'translation'}) != None :
        return remove_bg(soup.find('div', {'id': 'translation'}).find('p').get_text('\n', strip=True)).strip('\n')
    else:
        return "NONE"

In [10]:
def purport(soup):
    if soup.find('div', {'id': 'commentary'}) != None :
        return '\n'.join([paragraph.get_text(strip=True) for paragraph in soup.find('div', {'id': 'commentary'}).find_all('p')]).strip('\n')
    else:
        return "NONE"

In [11]:
import requests
from bs4 import BeautifulSoup
def scrape_data(verse_id, chapter, verse):
    url = f"https://www.holy-bhagavad-gita.org/chapter/{chapter}/verse/{verse}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        id = verse_id
        chapter_id = chapter
        verse_org_dev = '\n'.join([paragraph.get_text(strip=True) for paragraph in soup.find('div', {'id': 'originalVerse'}).find_all('p')]).strip('\n') if soup.find('div', {'id': 'originalVerse'}) else 'NONE'
        verse_org_roman = soup.find('div', {'id': 'transliteration_wo_dia'}).find('p').get_text('\n', strip=True).strip('\n') if soup.find('div', {'id': 'transliteration_wo_dia'}) else 'NONE'
        
        chapter_number_en = str(chapter)
        verse_number_en = soup.find('span', class_='breadcrumb_last').get_text(strip=True).replace('Verse', '').strip('\n') if soup.find('span', class_='breadcrumb_last') else 'NONE'
        chapter_number_dev = convert_to_hindi_number(chapter_number_en)
        verse_number_dev = convert_to_hindi_number(verse_number_en)
        
#         verse_en =  soup.find('div', {'id': 'transliteration'}).find('p').get_text('\n', strip=True).strip('\n') if soup.find('div', {'id': 'transliteration'}) else 'NONE'
#         synonyms_en =  ("; ".join( f"{entry.get_text(strip=True)}—{entry.find_next('span', {'class': 'meaning'}).get_text(strip=True)}" for entry in soup.find('div', {'id': 'wordMeanings'}).find_all('a', {'class': 'word'}) )).strip('\n')
#         translation_en =  translation(soup)
#         purport_en =  purport(soup)
        
        
        response = requests.get(url + "/hi")
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        verse_hi = find_verse(soup)
        synonyms_hi = synonyms(soup)
        translation_hi = translation(soup)
#         purport_hi = purport(soup)
        
#         response = requests.get(url + "/te")
#         response.raise_for_status()
#         soup = BeautifulSoup(response.content, "html.parser")
#         verse_te = find_verse(soup)
#         synonyms_te = synonyms(soup)
#         translation_te = translation(soup)
#         purport_te = purport(soup)
        
#         response = requests.get(url + "/or")
#         response.raise_for_status()
#         soup = BeautifulSoup(response.content, "html.parser")
#         verse_or = find_verse(soup)
#         synonyms_or = synonyms(soup)
#         translation_or = translation(soup)
#         purport_or = purport(soup)
        
#         response = requests.get(url + "/gu")
#         response.raise_for_status()
#         soup = BeautifulSoup(response.content, "html.parser")
#         verse_gu = find_verse(soup)
#         synonyms_gu = synonyms(soup)
#         translation_gu = translation(soup)
#         purport_gu = purport(soup)
        
#         response = requests.get(url + "/ta")
#         response.raise_for_status()
#         soup = BeautifulSoup(response.content, "html.parser")
#         verse_ta = remove_numbered_strings(find_verse(soup))
#         synonyms_ta = synonyms(soup)
#         translation_ta = translation(soup)
#         purport_ta = purport(soup)
        
        
        data = {
            "id" : id,
            "chapter_id" : chapter_id,
            "verse_org_dev" : verse_org_dev,
            "verse_org_roman" : verse_org_roman,
            
            "chapter_number_en" : chapter_number_en,
            "verse_number_en" : verse_number_en,
            "chapter_number_dev" : chapter_number_dev,
            "verse_number_dev" : verse_number_dev,
            
            "verse_en" : "",
            "synonyms_en" : "",
            "translation_en" : "",
            "purport_en" : "",
            
            "verse_hi" : verse_hi,
            "synonyms_hi" : synonyms_hi,
            "translation_hi" : translation_hi,
            "purport_hi" : "",
            
            "verse_te" : "",
            "synonyms_te" : "",
            "translation_te" : "",
            "purport_te" : "",
            
            "verse_or" : "",""
            "synonyms_or" : "",
            "translation_or" : "",
            "purport_or" : "",
            
            "verse_gu" : "",
            "synonyms_gu" : "",
            "translation_gu" : "",
            "purport_gu" : "",
            
            "verse_ta" : "",
            "synonyms_ta" : "",
            "translation_ta" : "",
            "purport_ta" : "", 
            
            "like" : False
        }
        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [12]:
verses_per_chapter = [47, 72, 43, 42, 29, 47, 30, 28, 34, 42, 55, 20, 35, 27, 20, 24, 28, 78]

In [13]:
# data = []

# verse_id = 1

# for chapter_number in range(1,19):
#     verse_count = verses_per_chapter[chapter_number-1]
#     for verse_number in range(1,verse_count+1):
#         verse = scrape_data(verse_id, chapter_number, verse_number)
#         if len(data) > 0 :
#             last_verse = data[len(data)-1]
#             if(last_verse['verse_number_en'] == verse['verse_number_en']):
#                 continue
#         data.append(verse)
#         verse_id += 1
#         print("verse " + str(verse['verse_number_en']))

In [14]:
# import json
# # Write the result to result.json
# with open('verses.json', 'w', encoding='utf-8') as file:
#     json.dump(data, file, ensure_ascii=False, indent=2)

In [15]:
# data2 = []
# verse_id = 580
# for chapter_number in range(18,19):
#     verse_count = verses_per_chapter[chapter_number-1]
#     for verse_number in range(15,verse_count+1):
#         verse = scrape_data(verse_id, chapter_number, verse_number)
#         if len(data2) > 0 :
#             last_verse = data2[len(data2)-1]
#             if(last_verse['verse_number_en'] == verse['verse_number_en']) :
#                 continue
#         data2.append(verse)
#         verse_id += 1
#         print("verse " + str(verse['verse_number_en']))

In [16]:
verses = [
    [1,9],
    [1,10],
    [1,19],
    [11,19],
    [13,1]
]

In [21]:
res = scrape_data(1,1,19)
res

{'id': 1,
 'chapter_id': 1,
 'verse_org_dev': 'स घोषो धार्तराष्ट्राणां हृदयानि व्यदारयत् |\nनभश्च पृथिवीं चैव तुमुलोऽभ्यनुनादयन् || 19||',
 'verse_org_roman': 'sa ghosho dhartarashtranam hridayani vyadarayat\nnabhashcha prithivim chaiva tumulo abhyanunadayan',
 'chapter_number_en': '1',
 'verse_number_en': ' 19',
 'chapter_number_dev': '१',
 'verse_number_dev': ' १९',
 'verse_en': '',
 'synonyms_en': '',
 'translation_en': '',
 'purport_en': '',
 'verse_hi': 'स घोषो धार्तराष्ट्राणां हृदयानि व्यदारयत्।\nनभश्च पृथिवीं चैव तुमुलोऽभ्यनुनादयन् ॥19॥',
 'synonyms_hi': 'सः-उस; घोषः-शब्द ध्वनि; धार्तराष्ट्राणाम् धृतराष्ट्र के पुत्रों के; हृदयानि- हृदयों को; व्यदारयत्-विदीर्ण कर दिया; नभ:-आकाश; च-भी; पृथिवीम्-पृथ्वी को; च-भी; एव-निश्चय ही; तुमुल:-कोलाहलपूर्ण ध्वनि; व्यनुनादयन्–गर्जना करना।',
 'translation_hi': 'हे धृतराष्ट्र! इन शंखों से उत्पन्न ध्वनि द्वारा आकाश और धरती के बीच हुई गर्जना ने\r\nआपके पुत्रों के हृदयों को विदीर्ण कर दिया।',
 'purport_hi': '',
 'verse_te': '',
 'synonyms_te': '',
 